In [6]:
# the imports used in A2
import re
# import json
from glob import glob
import os
from io import StringIO
from itertools import groupby
import pickle

import numpy as np
import bs4
%matplotlib inline
import matplotlib.pyplot as plt
# Imports that might help with various functionality
import functools
import operator

# Additional imports from A3
from __future__ import print_function
import math
from collections import defaultdict
from nltk.tokenize import TreebankWordTokenizer
import Levenshtein  # package python-Levenshtein

In [7]:
import sys
# Ensure that your kernel is using Python3
assert sys.version_info.major == 3

In [2]:
# import the json
import json
with open("updated_data.json", "r") as f:
    women_summaries = json.load(f)


In [3]:
# creates a list of women to keep as a global variable
women_names = list()
for i in range(len(women_summaries)):
    name = women_summaries[i]['name']
    end_index = name.find('(')
    women_names.append(name[: end_index])
    

In [4]:
# creates a dictionary {'name', [list of words in the summary]}
women_dict_1sent = dict()
for i in range(len(women_summaries)):
    name = women_summaries[i]['name']
    summary = women_summaries[i]['summary']
    # to do: 
    # hardcoded the words that we're looking for, 
    index_was = summary.find('was')
    index_is = summary.find('is')
    index_currently = summary.find('currently')
    if (index_was == -1 and index_is == -1):
        summary2 = summary[index_currently:]
        index_period = summary2.find('.')
        first_sent = summary[index_currently:index_period]
    elif (index_was == -1 and index_currently == -1):
        summary2 = summary[index_is:]
        index_period = summary2.find('.')
        first_sent = summary[index_is:index_period]
    elif (index_is == -1 and index_currently == -1):
        summary2 = summary[index_was:]
        index_period = summary2.find('.')
        first_sent = summary[index_was:index_period]
    else :
        index_min = min(index_is, index_was)
        print(index_min)
        summary2 = summary[index_min:]
        index_period = summary2.find('.')
        print(index_period)
        first_sent = summary[index_min:index_period]
    print(first_sent)

41
252
was an English fossil collector, dealer, and paleontologist who became known around the world for important finds she made in Jurassic marine fossil beds in the cliffs along the English Channel at Lyme Regis in 
63
26

78
121
is an American author who, from 1957 to 196
10
210
ise Carson (May 27, 1907 – April 14, 1964) was an American marine biologist, author, and conservationist whose book Silent Spring and other writings are credited with advancing the global environmenta
62
139
was a German-Canadian metallurgist, research physicist, author, and educator 
55
107
was an American scientist and cytogeneticist who was
56
90
was an Anglo-Indian Egyptologist, 
70
107
was a German mathematician known for 
92
149
was a British stateswoman who served as Prime Minister of
75
80
is on
90
250
was a British geologist, academic, and physicist, whose wartime work with radar and radio led to early developments in radio astronomy and whose post-war work o
62
91
was an American toxicologist 
57


ison (July 17, 1855 – 1933) was a linguist, artifact collector, and community leader from the Tsimshian First Nation of northwestern British Columb
is a Panamanian–American academic anthropologist and archaeologist, and senior staff scientist (emerita) at the Smithsonian Tropical Rese
is an American anthropologist at the University of Ca
48
101
was an Australian botanical illustrator, anthropologi
16
128
is (26 August 1948 – 9 April 2009) was a British social anthropologist whose work focused on the study of the Bo
9
83
wasser is an Israeli Egyptologist, professor of Egyptology at the Hebrew U
is a social anthropologist specialising in gender and Spanis

20
98
is a Canadian linguist specializing in phonology and known for her work on Fir
was a scholar committed to the study of the interaction of social behavior and physio
is 
is an Associate Professor at the University of North Carolina
is an American 
17
130
is an anthropologist and professor at the University of California, Santa Cru

In [5]:
# The file is ordered from the homeworks that we've worked on
# so that we can easily go back to the specific assignment if we need to

In [ ]:
# Tokenize the summaries so that we can compare words easier
# this currently would summarize the whole json file but we probably should 
# restrict it to the first sentence like Charles mentioned when we met him
# Adapted from homework 1

def tokenize_summary(text):
    lowercase = text.lower()
    lower_array = re.findall("[a-z]+", lowercase)
    return lower_array

In [ ]:
# tokenize the json file
# adapted from homework 1

def tokenize_json(file):
    # a for loop that goes through each of the people
    
# maybe store it as a dictionary {"woman", ["she", "was", "a", "palenotologist"]}
    

In [ ]:
# Create a matrix using jaccard similarity
def create_jaccard_sim_matrix(num_women, input_array, unique_words):
    arr = np.zeroes(shape = (num_women, num_women))
    for (i, woman1) in enumerate(input_array):
        for (j, woman2) in enumerate(input_array):
            s1 = np.nonzero(woman1)
            s2 = np.nonzero(woman2)
            intersect = np.intersect1d(s1, s2)
            union = np.union1d(np.array(s1).flatten, np.array(s2).flatten())
            arr[i][j] = len(intersect)/len(union)
    arr.diagonals() = 0
    return arr

In [ ]:
# we need to figure out an input_array for the Jaccard one
# we also need to figure out what we're considering unique words

In [ ]:
def sim_dict(input_sim_matrix, num_women):
    womenDict = dict()
    for (woman1) in enumerate (input_sim_matrix):
        for (woman2) in enumerate (input_sim_matrix):
            womenDict[(woman1, woman2)] = input_sim_matrix[woman1, woman2]
    return womenDict
            

In [ ]:
# Homework 3 looked at Levenshtein distance (probably not a good idea imo)

In [ ]:
# Homework 4 looked at cosine similarity, which we can consider on top of jaccard